In [2]:
# Imports básicos
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier


In [9]:
# 📌 1. Importar bibliotecas
import pandas as pd
import json

# 📌 2. Carregar os arquivos JSON
with open("../data/raw/applicants.json", "r", encoding="utf-8") as f:
    applicants_data = json.load(f)

with open("../data/raw/prospects.json", "r", encoding="utf-8") as f:
    content = f.read()
    try:
        prospects_data = json.loads(content)
    except json.JSONDecodeError as e:
        print("Erro ao interpretar prospects.json:", e)
        prospects_data = []

with open("../data/raw/vagas.json", "r", encoding="utf-8") as f:
    vagas_data = json.load(f)

In [17]:
# 📌 2. Carregar applicants.json
with open('../data/raw/applicants.json', 'r', encoding='utf-8') as f:
    applicants_data = json.load(f)

df_applicants = pd.json_normalize(applicants_data)
print(f"✔ Applicants carregados: {df_applicants.shape[0]} registros")

# 📌 3. Carregar vagas.json
with open('../data/raw/vagas.json', 'r', encoding='utf-8') as f:
    vagas_data = json.load(f)

df_vagas = pd.json_normalize(vagas_data)
print(f"✔ Vagas carregadas: {df_vagas.shape[0]} registros")

# 📌 4. Carregar prospects.json (dict de vagas com prospects)
with open('../data/raw/prospects.json', 'r', encoding='utf-8') as f:
    prospects_data = json.load(f)

✔ Applicants carregados: 1 registros
✔ Vagas carregadas: 1 registros


In [18]:
df_applicants.head()

,31000.infos_basicas.telefone_recado,31000.infos_basicas.telefone,31000.infos_basicas.objetivo_profissional,31000.infos_basicas.data_criacao,31000.infos_basicas.inserido_por,31000.infos_basicas.email,31000.infos_basicas.local,31000.infos_basicas.sabendo_de_nos_por,31000.infos_basicas.data_atualizacao,31000.infos_basicas.codigo_profissional,...,5999.informacoes_profissionais.certificacoes,5999.informacoes_profissionais.outras_certificacoes,5999.informacoes_profissionais.remuneracao,5999.informacoes_profissionais.nivel_profissional,5999.formacao_e_idiomas.nivel_academico,5999.formacao_e_idiomas.nivel_ingles,5999.formacao_e_idiomas.nivel_espanhol,5999.formacao_e_idiomas.outro_idioma,5999.cv_pt,5999.cv_en
0,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,31000,...,,,0,,,,,-,,


In [19]:
# 📌 5. Normalizar dados de prospects com info da vaga
prospect_list = []

for vaga_id, vaga_info in prospects_data.items():
    titulo_vaga = vaga_info.get("titulo", "")
    modalidade = vaga_info.get("modalidade", "")
    
    for prospect in vaga_info.get("prospects", []):
        prospect["vaga_id"] = vaga_id
        prospect["titulo_vaga"] = titulo_vaga
        prospect["modalidade"] = modalidade
        prospect_list.append(prospect)

df_prospects = pd.json_normalize(prospect_list)
print(f"✔ Prospects normalizados: {df_prospects.shape[0]} registros")

✔ Prospects normalizados: 53759 registros


In [ ]:
print("📋 Colunas em df_applicants:", df_applicants.columns.tolist())


In [20]:
# 📌 6. Padronizar o tipo de código para merge
df_applicants["codigo"] = df_applicants["codigo"].astype(str)
df_prospects["codigo"] = df_prospects["codigo"].astype(str)
df_vagas["id"] = df_vagas["id"].astype(str)

KeyError: 'codigo'